In [1]:
with open("./input.txt", "r") as file: 
    data = [row.split(" ") for row in file.read().strip().split("\n")]

# Part 1

In [2]:
from enum import Enum

class OUTCOMES(Enum):
    WIN  = "WIN"
    DRAW = "DRAW"
    LOSE = "LOSE"
    
class HANDS(Enum):
    ROCK     = "ROCK"
    PAPER    = "PAPER"
    SCISSORS = "SCISSORS"
    
class MAPPING(Enum):
    A = HANDS.ROCK
    B = HANDS.PAPER
    C = HANDS.SCISSORS
    X = HANDS.ROCK
    Y = HANDS.PAPER
    Z = HANDS.SCISSORS
    
class POINTS(Enum):
    ROCK     = 1
    PAPER    = 2
    SCISSORS = 3
    WIN      = 6
    DRAW     = 3
    LOSE     = 0
    
def play(hands:tuple):
    """
    Returns outcome from a game of "rock, paper, scissors"
    
    Parameters
    ----------
    hands : tuple
        Tuple of plays (A, Y)
    """
    if hands[0] == hands[1]: 
        return OUTCOMES.DRAW
    
    if (hands[0] == HANDS.ROCK and hands[1] == HANDS.PAPER) or \
        (hands[0] == HANDS.PAPER and hands[1] == HANDS.SCISSORS) or \
        (hands[0] == HANDS.SCISSORS and hands[1] == HANDS.ROCK):
        return OUTCOMES.WIN
    
    if (hands[1] == HANDS.ROCK and hands[0] == HANDS.PAPER) or \
        (hands[1] == HANDS.PAPER and hands[0] == HANDS.SCISSORS) or \
        (hands[1] == HANDS.SCISSORS and hands[0] == HANDS.ROCK):
        return OUTCOMES.LOSE
    
    raise Exception(hands)

def score(hands:tuple):
    """
    Returns the score of a game of "rock, paper, scissors"
    
    Parameters
    ----------
    hands : tuple of enum types
        Tuple of plays (HANDS.ROCK, HANDS.PAPER)
    """
    outcome = play(
        (
            hands[0], 
            hands[1]
        )
    )

    return POINTS[outcome.name].value + POINTS[hands[1].value].value

def main(data):
    return sum(score((MAPPING[hands[0]].value, MAPPING[hands[1]].value)) for hands in data)

main(data)

12645

# Part 2

In [3]:
class TARGET_OUTCOMES(Enum):
    X = OUTCOMES.LOSE
    Y = OUTCOMES.DRAW
    Z = OUTCOMES.WIN
    
def reverse(opponent_hand, target_outcome):
    if target_outcome == OUTCOMES.DRAW: 
        return opponent_hand
    
    if target_outcome == OUTCOMES.WIN: 
        if opponent_hand == HANDS.ROCK:
            return HANDS.PAPER
        if opponent_hand == HANDS.PAPER: 
            return HANDS.SCISSORS
        if opponent_hand == HANDS.SCISSORS:
            return HANDS.ROCK
        raise ValueError(f"Unexpected opponent hand {opponent_hand}")
        
    if target_outcome == OUTCOMES.LOSE: 
        if opponent_hand == HANDS.ROCK:
            return HANDS.SCISSORS
        if opponent_hand == HANDS.PAPER: 
            return HANDS.ROCK
        if opponent_hand == HANDS.SCISSORS:
            return HANDS.PAPER
        raise ValueError(f"Unexpected opponent hand {opponent_hand}")
    
    raise ValueError(f"Unexpected target outcome {target_outcome}")
    
def main(data): 
    return sum(
        score((
            MAPPING[hands[0]].value, 
            reverse(MAPPING[hands[0]].value, TARGET_OUTCOMES[hands[1]].value)
        )) 
        for hands in data
    )

main(data)

11756

# Illustrations

In [4]:
import easychart
import pandas as pd

df = pd.DataFrame(data, columns=["opponent","me"]).apply(
    lambda col: col.map({
        "A":"Rock",
        "B":"Paper",
        "C":"Scissors",
        "X":"Rock",
        "Y":"Paper",
        "Z":"Scissors"
    })
)

outcomes = {
    ("Rock","Paper"): "Win",
    ("Paper", "Scissors"): "Win",
    ("Scissors", "Rock"): "Win"
}

df["outcome"] = df.apply(
    lambda row: 
        "Draw" if row["opponent"] == row["me"] 
        else outcomes.get(tuple(row), "Lose"),
    axis=1
)

df.head()

,opponent,me,outcome
0,Paper,Scissors,Win
1,Rock,Paper,Win
2,Rock,Paper,Win
3,Paper,Rock,Lose
4,Rock,Paper,Win


In [5]:
plotdata = df.groupby(["outcome"])["me"].value_counts().unstack()
plotdata

me,Paper,Rock,Scissors
outcome,,,
Draw,28,533,89
Lose,275,440,128
Win,460,69,478


In [6]:
chart = easychart.new("pie")
chart.title = "Number of played games, by outcome"
chart.plot(plotdata.sum(axis=1), labels="{point.y} {point.name}s ({point.percentage:.0f}%)")
chart

Tree({'series': [{'data': [['Draw', 650], ['Lose', 843], ['Win', 1007]], 'dataLabels': {'enabled': True, 'format': '{point.y} {point.name}s ({point.percentage:.0f}%)'}, 'name': None}], 'chart': {'type': 'pie', 'zoomType': 'x'}, 'title': {'text': 'Number of played games, by outcome'}})

In [7]:
chart = easychart.new("column")
chart.title = "Played hand by outcome"
chart.categories = plotdata.index
chart.stacked = "percent"
chart.yAxis.labels.format = "{value}%"

for play in plotdata: 
    chart.plot(plotdata[play])
    
chart

Tree({'series': [{'data': [['Draw', 28], ['Lose', 275], ['Win', 460]], 'name': 'Paper'}, {'data': [['Draw', 533], ['Lose', 440], ['Win', 69]], 'name': 'Rock'}, {'data': [['Draw', 89], ['Lose', 128], ['Win', 478]], 'name': 'Scissors'}], 'chart': {'type': 'column', 'zoomType': 'x'}, 'title': {'text': 'Played hand by outcome'}, 'xAxis': {'categories': Index(['Draw', 'Lose', 'Win'], dtype='object', name='outcome')}, 'plotOptions': {'series': {'stacking': 'percent'}}, 'yAxis': {'labels': {'format': '{value}%'}}})